### classify stackoverflow posts by considering each document a simple mean of its individual word embeddings

**TODO**
 - use weighted averages (tf-id, ppmi, etc)
 - use a different scoring function so that wrong guesses that are close to the correct values (think "python", "python-2") are penalized less than guesses that are wildly wrong.


In [1]:
import csv
import os
import re
import sys
import numpy as np
import sklearn

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Input, Flatten, Activation
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalMaxPooling1D
from keras.models import Model, Sequential

from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer


# my stuff in the helpers/ directory
from helpers import files,texts, metrics,tags

Using TensorFlow backend.


In [2]:
np.random.seed(seed=42)

In [3]:
texts, labels = files.read_stackoverflow_sample_small()

In [4]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM=100

LABELS_MIN_DOC_COUNT = int(len(texts)/3000)

In [5]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index

In [6]:
truncated_labels = tags.truncate_labels(labels,LABELS_MIN_DOC_COUNT)

In [7]:
lb = preprocessing.MultiLabelBinarizer()
binary_labels = lb.fit_transform(truncated_labels)


# tag position => fraction of docs having that tag
# TODO use just the training examples otherwise it's snooping
tag_probabilities_index = tags.get_probabilities_index(binary_labels)

In [8]:
# word => embedding
embeddings_index = files.read_glove_100()

In [9]:
# word_position => embedding for that word
numeric_embeddings_index = {}

embedding_matrix = np.zeros((len(word_index)+1,EMBEDDING_DIM))

for word,i in word_index.items():
    
    if i >= MAX_NB_WORDS:
        continue  
    
    embedding_vector = embeddings_index.get(word)
    
    numeric_embeddings_index[i] = embedding_vector
    
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [10]:
inverse_word_index = {}

for word,i in word_index.items():
    inverse_word_index[i] = word

In [11]:
document_embeddings = []

for i,document_sequence in enumerate(sequences):
    
    individual_word_embeddings = []
    
    for word_position in document_sequence:       
                                  
        word = inverse_word_index[word_position]
        
        word_embedding = embeddings_index.get(word)    

        if word_embedding is None:
            continue
        else:    
            individual_word_embeddings.append(word_embedding)
         
    if len(individual_word_embeddings) == 0:
        # some documents have no text
        # so we assign a zero vector
        individual_word_embeddings = np.zeros(
            EMBEDDING_DIM,dtype='float32').reshape(1,EMBEDDING_DIM)
        
    document_embedding = np.mean(individual_word_embeddings,axis=0)
    
    document_embeddings.append(document_embedding)

In [12]:
data = np.vstack(document_embeddings)
indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
labels_1 = binary_labels[indices]

num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

X_train = data[:-num_validation_samples]
Y_train = labels_1[:-num_validation_samples]
X_val = data[-num_validation_samples:]
Y_val = labels_1[-num_validation_samples:]

data.shape,labels_1.shape

((22000, 100), (22000, 1224))

In [13]:
%%time

num_labels = labels_1.shape[1]

model = Sequential()
model.add(Dense(512,input_dim=EMBEDDING_DIM))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(512))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(num_labels))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam')
# multi label needs a specific accuracy function

model.fit(X_train, Y_train,
                    batch_size=32,
                    epochs=10,
                    verbose=1,
                    validation_data=(X_val,Y_val))

Train on 17600 samples, validate on 4400 samples
Epoch 1/10
17600/17600 [==============================] - 3s - loss: 0.0193 - val_loss: 0.0113
Epoch 2/10
17600/17600 [==============================] - 2s - loss: 0.0116 - val_loss: 0.0111
Epoch 3/10
17600/17600 [==============================] - 3s - loss: 0.0113 - val_loss: 0.0109
Epoch 4/10
17600/17600 [==============================] - 3s - loss: 0.0111 - val_loss: 0.0106
Epoch 5/10
17600/17600 [==============================] - 2s - loss: 0.0108 - val_loss: 0.0104
Epoch 6/10
17600/17600 [==============================] - 2s - loss: 0.0106 - val_loss: 0.0102
Epoch 7/10
17600/17600 [==============================] - 2s - loss: 0.0104 - val_loss: 0.0101
Epoch 8/10
17600/17600 [==============================] - 3s - loss: 0.0103 - val_loss: 0.0099
Epoch 9/10
17600/17600 [==============================] - 3s - loss: 0.0101 - val_loss: 0.0099
Epoch 10/10
17600/17600 [==============================] - 2s - loss: 0.0100 - val_loss: 0.0098


In [71]:
## DEBUG

num_test_cases = 10
THRESHOLD = 45.0

relative_difference_threshold = THRESHOLD

for i in np.random.randint(low=0, high=len(Y_val), size=num_test_cases):

    actual_label_indices = Y_val[i].reshape(1,-1)
    predicted_tag_probabilities = model.predict(X_val[i].reshape(1,-1))
    
    predicted_label_indices = tags.get_predicted_indices(tag_probabilities_index,
                                                        predicted_tag_probabilities,
                                                        relative_difference_threshold)

    actual_labels = lb.inverse_transform(actual_label_indices)
    
    predicted_label_scores = predicted_tag_probabilities[0][predicted_label_indices.ravel().nonzero()[0]]
    predicted_labels = lb.inverse_transform(predicted_label_indices)

    tags_and_scores = sorted(list(zip(predicted_labels[0],predicted_label_scores)), key=lambda tpl: tpl[1], reverse=True)
    
    print(actual_labels)
    print(tags_and_scores)
    print('\n')

[('google-play',)]
[('sql', 0.23382163), ('oracle', 0.04773559), ('tsql', 0.025429932), ('sql-order-by', 0.0043521612), ('generator', 0.0042944308), ('metadata', 0.0029409875)]




In [22]:
X_val.shape

(4400, 100)

In [23]:
X_test_list = []

for validation_features in X_val:
    X_test_list.append(validation_features.reshape(1,-1))

X_test = np.vstack(X_test_list)    

In [24]:
Y_pred_lst = [model.predict(x_test.reshape(1,-1)) for x_test in X_test]

In [25]:
Y_pred_calculated_lst = [tags.get_predicted_indices(tag_probabilities_index,y_pred,THRESHOLD) for y_pred in np.vstack(Y_pred_lst)]

Y_pred = np.vstack(Y_pred_calculated_lst)

Y_val.shape,Y_pred.shape

((4400, 1224), (4400, 1224))

In [26]:
micro_f1 = metrics.calculate_multilabel_metrics(Y_val,Y_pred)
print('micro-averaged F1 score (validation set) is {0}'.format(micro_f1))

micro_prec = metrics.calculate_multilabel_metrics(Y_val, Y_pred, metric='precision')
print('micro-averaged precision score (validation set) is {0}'.format(micro_prec))

micro_rec = metrics.calculate_multilabel_metrics(Y_val, Y_pred, metric='recall')
print('micro-averaged recall score (validation set) is {0}'.format(micro_rec))


micro-averaged F1 score (validation set) is 0.04015526175792564
micro-averaged precision score (validation set) is 0.025061485489424495
micro-averaged recall score (validation set) is 0.10096106212226295
